## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

D:\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, (3, 3), padding = 'same', activation = 'relu', input_shape = (32, 32, 3)))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(BatchNormalization())


#卷積組合
classifier.add(Convolution2D(64, (3, 3), padding = 'same', activation = 'relu'))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim = 100, activation = 'relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100)`
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
50000/50000 [==============================] - 25s 502us/step - loss: 1.3869 - acc: 0.5289
Epoch 2/100
50000/50000 [==============================] - 22s 448us/step - loss: 0.8901 - acc: 0.6865
Epoch 3/100
50000/50000 [==============================] - 22s 447us/step - loss: 0.6563 - acc: 0.76838s - loss: 0.642
Epoch 4/100
50000/50000 [==============================] - 22s 448us/step - loss: 0.4625 - acc: 0.8375- ETA: 1s - loss: 0.4597 -
Epoch 5/100
50000/50000 [==============================] - 22s 443us/step - loss: 0.3145 - acc: 0.89190s - loss: 0.3121 - ac
Epoch 6/100
50000/50000 [==============================] - 22s 448us/step - loss: 0.2142 - acc: 0.9262
Epoch 7/100
50000/50000 [==============================] - 22s 448us/step - loss: 0.1612 - acc: 0.9455
Epoch 8/100
50000/50000 [==============================] - 22s 449us/step - loss: 0.1302 - acc: 0.9548
Epoch 9/100
50000/

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[2.1214985e-06, 8.8122970e-06, 3.0843496e-06, 9.9505323e-01,
        4.9327831e-03, 5.0912754e-09, 8.1658389e-13, 6.1654312e-08,
        1.2705424e-09, 3.4285086e-13]], dtype=float32)